In [ ]:
class Matrix:
    def __init__(self, rows):
        self.data = rows  # Initialize with a list of lists
        self.flops = 0
        self.cols = len(rows)
        self.rows = len(rows[0])
    def __getitem__(self, index):
        # Return the row at the given index
        return self.data[index]

    def __setitem__(self, index, value):
        # Set the row at the given index
        self.data[index] = value

    def __repr__(self) -> str:
        output = ''
        for row in self.data:
            output+="|"
            for elem in row:
                output+=' '+"%.16f" %elem+' '
            output+='|\n'
        return output

    def swapItems(self,item1,item2):
        temp = self.data[item1[0]][item1[1]]
        self.data[item1[0]][item1[1]] = self.data[item2[0]][item2[1]]
        self.data[item2[0]][item2[1]] = self.data[item1[0]][item1[1]]

    def swapCols(self,index1,index2):
        for row in self.data:
            temp = row[index1]
            row[index1] = row[index2]
            row[index2] = temp

    def swapRows(self,idx1,idx2):
        temp = self.data[idx1]
        self.data[idx1] = self.data[idx2]
        self.data[idx2] = temp

    def mulRow(self,idx,k):
        for i in range(len(self.data[idx])):
            self.data[idx][i] *= k
            self.flops+=1

    def divRow(self,idx,k):
        self.mulRow(idx,1/k)
    
    def mulCol(self,idx,k):
        for row in self.data:
            row[idx] *=k
            self.flops+=1

    def divCol(self,idx,k):
        self.mulCol(idx,1/k)

    def AddRow(self,idx1,idx2):
        for i, item in enumerate(self.data[idx2]):
            self.data[idx1][i] += item
            self.flops+=1

    def SubRow(self,idx1,idx2):
        for i, item in enumerate(self.data[idx2]):
            self.data[idx1][i] -= item
            self.flops+=1

    def gauss(self):
        for i in range(len(self.data)-1):
            for j in range(i+1,len(self.data)):
                self.mulRow(j, self.data[i][i]/self.data[j][i])
                self.SubRow(j,i)

    def PPgauss(self):
        for i in range(len(self.data)-1):
            for j in range(i+1,len(self.data)):
                self.mulRow(j, self.data[i][i]/self.data[j][i])
                self.SubRow(j,i)

    def reverse(self):
        for i in range(len(self.data)-1,0):
            for j in range(i+1,len(self.data)):
                self.mulRow(j, self.data[i][i]/self.data[j][i])
                self.SubRow(j,i)




# Create a 2D matrix
matrix = Matrix([
    [0.00000000000001,4,8],
    [1, -1, 1]
])
print("matrix without macheps\n",matrix)

# Accessing elements using [][] works naturally

matrix.gauss()
print(matrix)
matrix.mulRow(0,matrix[1][1]/matrix[0][1])
matrix.SubRow(0,1)
print(matrix)
matrix.divRow(0,matrix[0][0])
print(matrix)
matrix.divRow(1,matrix[1][1])
print(matrix)

matrix2 = Matrix([
    [0.0000000000000001,4,8],
    [1, -1, 1]
])
print("matrix with macheps\n",matrix2)

# Accessing elements using [][] works naturally

matrix2.gauss()
print(matrix2)
matrix2.mulRow(0,matrix2[1][1]/matrix2[0][1])
matrix2.SubRow(0,1)
print(matrix2)
matrix2.divRow(0,matrix2[0][0])
print(matrix2)
matrix2.divRow(1,matrix2[1][1])
print(matrix2)

matrix without macheps
 | 0.0000000000000100  4.0000000000000000  8.0000000000000000 |
| 1.0000000000000000  -1.0000000000000000  1.0000000000000000 |

| 0.0000000000000100  4.0000000000000000  8.0000000000000000 |
| 0.0000000000000000  -4.0000000000000098  -7.9999999999999902 |

| -0.0000000000000100  0.0000000000000000  -0.0000000000000293 |
| 0.0000000000000000  -4.0000000000000098  -7.9999999999999902 |

| 1.0000000000000000  -0.0000000000000000  2.9309887850104066 |
| 0.0000000000000000  -4.0000000000000098  -7.9999999999999902 |

| 1.0000000000000000  -0.0000000000000000  2.9309887850104066 |
| -0.0000000000000000  1.0000000000000000  1.9999999999999927 |

matrix with macheps
 | 0.0000000000000001  4.0000000000000000  8.0000000000000000 |
| 1.0000000000000000  -1.0000000000000000  1.0000000000000000 |

| 0.0000000000000001  4.0000000000000000  8.0000000000000000 |
| 0.0000000000000000  -4.0000000000000000  -8.0000000000000000 |

| -0.0000000000000001  0.0000000000000000  0.000000